In [ ]:
import os

#we first get the absolute paths of every positive and negative datasets so we can then read them

#edX dataset
#find all the positive datasets
edx_positives  = os.popen("find ./drive/My\ Drive/edX-Udacity-dataset/edX-prerequisites/edX-prerequisites -name 'positive.txt'").read().split('\n')
#find all the negative datasets
edx_negatives = os.popen("find ./drive/My\ Drive/edX-Udacity-dataset/edX-prerequisites/edX-prerequisites -name 'negative.txt'").read().split('\n')

#Udacity
#find all the positive datasets
udacity_positives = os.popen("find ./drive/My\ Drive/edX-Udacity-dataset/Udacity-prerequisites/Udacity-prerequisites -name 'positive.txt'").read().split('\n')
#find all the negative datasets
udacity_negatives = os.popen("find ./drive/My\ Drive/edX-Udacity-dataset/Udacity-prerequisites/Udacity-prerequisites -name 'negative.txt'").read().split('\n')

#list of all the absolute paths of the positive and negative datasets
#we also pop the last element since due to the split done before it's just an empty string
positives = edx_positives[:-1] + udacity_positives[:-1]
negatives = edx_negatives[:-1] + udacity_negatives[:-1]


df_positives = pd.DataFrame() 
df_negatives = pd.DataFrame() 
#we now create a dataset for all the positive and negative pairs
#the elements of the pair are the filename of the referenced article, not the article itself
for path in positives:
  temp_df = pd.read_csv(path, sep='\t', header=None, warn_bad_lines=True, error_bad_lines=False)
  df_positives = pd.concat([df_positives,temp_df])

for path in negatives:
  temp_df = pd.read_csv(path, sep='\t', header=None, warn_bad_lines=True, error_bad_lines=False)
  df_negatives = pd.concat([df_negatives,temp_df])

#we obtain the negative counterpart of the positive dataset because:
#if B is prereq of A, then A is not prereq of B
cols = list(df_positives.columns)
cols = cols[1::2] + cols[::2]
neg = df_positives[cols]
#we concat it to the other negatives dataset
df_negatives = pd.concat([df_negatives,neg])

#we now assign the third column that says if there's a prerequisite or not
#1 for positive
#0 for negative
df_positives['2'] = '1'
df_negatives['2'] = '0'

#finally we concat everything into an unique dataset
df = pd.concat([df_positives,df_negatives])
#we shuffle the dataframe since right now it would give trouble while creating a proper training and validation set
df = df.sample(frac=1)

#train and validatin split
train = df.sample(frac=0.8,random_state=200) 
validation = df.drop(train.index)
print(df)

In [ ]:
#create dictionary of tokenized documents
nltk.download('punkt')
nltk.download('stopwords')

punct = string.punctuation + '«``»' + "''"
en_stop_words = nltk.corpus.stopwords.words('english')

doc_dict = {}

#we get the pages related to the filenames in the

for index,row in df.iterrows():
  #Titles of the two documents
  filename_A = row[0]
  filename_B = row[1]
  
  #we check if the page related to the titles is already in the dictionary
  if filename_A not in doc_dict:
    #we get the path of the page
    #there can be multiple results, they're all the same (there are copies of the same file)
    #we just get the first
    path = os.popen("find ./drive/My\ Drive/edX-Udacity-dataset -name '" + filename_A + "'").read().split('\n')[0]
    # print("--------------------")
    # print(os.popen("find ./drive/My\ Drive/edX-Udacity-dataset -name '" + filename_A + "'").read())
    # print(filename_A)
    page_file = open(path, "r")
    page = page_file.read()
    page_file.close()
    #clean the page 
    #document to lowercase, replaced apostrophe with space since the tokenizer isn't able to split words like "l'addizione"
    document = page.replace("'"," ")
    document = document.replace("\\"," ")
    document = document.replace("displaystyle"," ")
    #tokenized
    tokenized = nltk.tokenize.word_tokenize(document, "english")
    #no punctuation and lowercase
    no_punct = [x.lower() for x in tokenized if x not in punct]
    #remove stop words
    no_stop_words = [x for x in no_punct if x not in en_stop_words]
    #add tokenized document
    doc_dict[filename_A] = no_stop_words

  if filename_B not in doc_dict:
    #we get the path of the page
    #there can be multiple results, they're all the same (there are copies of the same file)
    #we just get the first
    path = os.popen("find ./drive/My\ Drive/edX-Udacity-dataset -name '" + filename_B + "'").read().split('\n')[0]
    page_file = open(path, "r")
    page = page_file.read()
    page_file.close()
    #clean the page 
    #document to lowercase, replaced apostrophe with space since the tokenizer isn't able to split words like "l'addizione"
    document = page.replace("'"," ")
    document = document.replace("\\"," ")
    document = document.replace("displaystyle"," ")
    #tokenized
    tokenized = nltk.tokenize.word_tokenize(document, "english")
    #no punctuation and lowercase
    no_punct = [x.lower() for x in tokenized if x not in punct]
    #remove stop words
    no_stop_words = [x for x in no_punct if x not in en_stop_words]
    #add tokenized document
    doc_dict[filename_B] = no_stop_words 